# Binomial on 2D data

Our model will be two binomial distribution per bin on a 2D grid. One dimension will be the distance to the sea (i.e. longitude if the sea shore is North-South) and the second would be distance to the river (i.e. latitude following the provious example). 

The probability of sucess of the binomial distribution will come from a $\beta(a,b)$ distribution. Spatial information is relevant because the $a$ will only vary with the river distance and $b$ will only vary with sea distance.

Therefore, we have a grid ${{x_i, y_j}} \forall i=1:N, j=1:M$ , where each $x_i, y_j$ pair (district) has 2 data values, the total number of votes and the number of votes of the right wing party (it is a 2 party political system, thus, total-right=left wing votes).

It does not have much sense, but we know that both variables follow a binomial distribution $votes(x,y) \sim B\Big(N_{inhabitants}, \frac{0.9}{1+\text{Beta}\big(\alpha(x),\beta(y)\big)}\Big)$ and $right(x,y) \sim B\Big(votes(x,y), \text{Beta}\big(\alpha(x),\beta(y)\big)\Big)$. Therefore, our model has $N$ _plus_ $M$ parameters, instead of the product that would be if each district was independent.

## Load data

In [1]:
import pystan
import pandas as pd
import numpy as np
import arviz as az
import matplotlib.pyplot as plt

In [2]:
N_inhabitants = 26000
data = pd.read_csv("2D_data_N_inhabitants_{}.csv".format(N_inhabitants)).set_index(["category","number"])
Total = data.loc["total"].values
Right = data.loc["right"].values
N, M = Total.shape

In [3]:
N,M

(13, 8)

In [4]:
binomial_on_2D_dat = {
    'N': N,
    'M': M,
    "N_inhabitants": N_inhabitants,
    'Total': Total,
    'Right': Right,
}

## PyStan code

In [5]:
binomial_on_2D_code = """
data {
    int<lower=1> N;     // num of x, or num of river_distance values
    int<lower=1> M;     // num of y, or num of sea_distance values
    int<lower=1> N_inhabitants;

    int Total[N,M];
    int Right[N,M];
}

parameters {
    vector<lower=0>[N] alphas;     
    vector<lower=0>[M] betas;
    real<lower=0, upper=1> p_intention[N,M];
    real<lower=0, upper=1> p_aux[N,M];
}

transformed parameters {
    real<lower=0, upper=1> p_participation[N,M];
    
    for (n in 1:N){
        for (m in 1:M){
            p_participation[n,m] = .9/(1+p_aux[n,m]);
        }
    }
    
}



model {

    for (n in 1:N){
        for (m in 1:M){
            p_intention[n,m] ~ beta(alphas[n], betas[m]);
            p_aux[n,m] ~ beta(alphas[n], betas[m]);
            Total[n,m] ~ binomial(N_inhabitants, p_participation[n,m]);
            Right[n,m] ~ binomial(Total[n,m], p_intention[n,m]);
        }
    }
}

generated quantities {
    real log_lik[N,M];
    real Total_hat[N,M];
    real Right_hat[N,M];
    
    for (n in 1:N){
        for (m in 1:M){
            log_lik[n,m] = binomial_lpmf(Total[n,m] | N_inhabitants, p_participation[n,m]) + 
                           binomial_lpmf(Right[n,m] | Total[n,m], p_intention[n,m]);
            Total_hat[n,m] = binomial_rng(N_inhabitants, p_participation[n,m]);
            Right_hat[n,m] = binomial_rng(Total[n,m], p_intention[n,m]);
        }
    }
}
"""

In [6]:
sm = pystan.StanModel(model_code=binomial_on_2D_code)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_a0be994a7f5f251f45b1e4dcc0dc0691 NOW.


In [7]:
fit = sm.sampling(
    data=binomial_on_2D_dat, 
    iter=2000, 
    chains=8, 
    pars=[
        'alphas', 
        'betas',
        'Total_hat',
        'Right_hat',
        'log_lik'
    ]
)

In [8]:
dims = {"alphas":["river_distance"], 
        "betas":["sea_distance"], 
        "p_intention": ["river_distance", "sea_distance"],  
        "p_participation": ["river_distance", "sea_distance"], 
        "Total": ["river_distance", "sea_distance"], 
        "Right": ["river_distance", "sea_distance"], 
        "Total_hat": ["river_distance", "sea_distance"], 
        "Right_hat": ["river_distance", "sea_distance"], 
        "log_lik": ["river_distance", "sea_distance"]}
coords = {"river_distance":range(N), "sea_distance": range(M)}
idata = az.from_pystan(
    posterior=fit,
    observed_data=['Total', 'Right'],
    posterior_predictive=['Total_hat', 'Right_hat'],
    log_likelihood="log_lik",
    coords=coords,
    dims=dims
)

In [9]:
az.loo(idata)

/home/oriol/venvs/arviz-dev/lib/python3.6/site-packages/arviz/stats/stats.py:425: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.7 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO posterior are very different. This is more likely to happen with a non-robust model and highly influential observations.
  "Estimated shape parameter of Pareto distribution is greater than 0.7 for "


loo           2586.42
loo_se        7.54498
p_loo         168.547
warning          True
loo_scale    deviance
dtype: object

In [10]:
idata.to_netcdf("binomial_on_2D_pystan.nc")

'binomial_on_2D_pystan.nc'

## Constant success probability model
Now the model will be $votes(x,y) = B(N_{inhabitants}, p_{participation})$ and $right(x,y) = B(votes(x,y), p_{intention})$, being $p_{participation}$ and $p_{intention}$ constants.

In [11]:
binomial_on_2D_code_constant = """
data {
    int<lower=1> N;     // num of x, or num of river_distance values
    int<lower=1> M;     // num of y, or num of sea_distance values
    int<lower=1> N_inhabitants;

    int Total[N,M];
    int Right[N,M];
}

parameters {
    real<lower=0, upper=1> p_intention;
    real<lower=0, upper=1> p_participation;
}

model {

    for (n in 1:N){
        for (m in 1:M){
            Total[n,m] ~ binomial(N_inhabitants, p_participation);
            Right[n,m] ~ binomial(Total[n,m], p_intention);
        }
    }
}

generated quantities {
    real log_lik[N,M];
    real Total_hat[N,M];
    real Right_hat[N,M];
    
    for (n in 1:N){
        for (m in 1:M){
            log_lik[n,m] = binomial_lpmf(Total[n,m] | N_inhabitants, p_participation) + 
                           binomial_lpmf(Right[n,m] | Total[n,m], p_intention);
            Total_hat[n,m] = binomial_rng(N_inhabitants, p_participation);
            Right_hat[n,m] = binomial_rng(Total[n,m], p_intention);
        }
    }
}
"""

In [12]:
sm_constant = pystan.StanModel(model_code=binomial_on_2D_code_constant)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_4e7c0de38032ecbed33eec128599cf4b NOW.


In [13]:
fit_constant = sm_constant.sampling(
    data=binomial_on_2D_dat, 
    iter=1000, 
    chains=4
)

In [14]:
dims = {"Total": ["river_distance", "sea_distance"], 
        "Right": ["river_distance", "sea_distance"], 
        "Total_hat": ["river_distance", "sea_distance"], 
        "Right_hat": ["river_distance", "sea_distance"], 
        "log_lik": ["river_distance", "sea_distance"]}
coords = {"river_distance":range(N), "sea_distance": range(M)}
idata_constant = az.from_pystan(
    posterior=fit_constant,
    observed_data=['Total', 'Right'],
    posterior_predictive=['Total_hat', 'Right_hat'],
    log_likelihood="log_lik",
    coords=coords,
    dims=dims
)

In [15]:
az.loo(idata_constant)

/home/oriol/venvs/arviz-dev/lib/python3.6/site-packages/arviz/stats/stats.py:425: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.7 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO posterior are very different. This is more likely to happen with a non-robust model and highly influential observations.
  "Estimated shape parameter of Pareto distribution is greater than 0.7 for "


loo            579789
loo_se        50832.2
p_loo         3059.64
warning          True
loo_scale    deviance
dtype: object

In [16]:
idata_constant.to_netcdf("binomial_on_2D_pystan_p_constant.nc")

'binomial_on_2D_pystan_p_constant.nc'

## Unknown number of inhabitants (version 1)
The third modelling option will be a variation on the first model, assuming that $N_{inhabitants}$ is not known, and modelling the participation as a poisson process. Hence, $votes(x,y) \sim \text{Poisson}(\lambda)$ and $right(x,y) \sim BetaBinomial\Big(votes(x,y), \alpha(x),\beta(y)\Big)$.

In [17]:
binomial_on_2D_code_unknown1 = """
data {
    int<lower=1> N;     // num of x, or num of river_distance values
    int<lower=1> M;     // num of y, or num of sea_distance values

    int Total[N,M];
    int Right[N,M];
}

parameters {
    vector<lower=0>[N] alphas;     
    vector<lower=0>[M] betas;
    real<lower=0> lambda;
}

model {

    for (n in 1:N){
        for (m in 1:M){
            Total[n,m] ~ poisson(lambda);
            Right[n,m] ~ beta_binomial(Total[n,m], alphas[n], betas[m]);
        }
    }
}

generated quantities {
    real log_lik[N,M];
    real Total_hat[N,M];
    real Right_hat[N,M];
    
    for (n in 1:N){
        for (m in 1:M){
            log_lik[n,m] = poisson_lpmf(Total[n,m] | lambda) + 
                           beta_binomial_lpmf(Right[n,m] | Total[n,m], alphas[n], betas[m]);
            Total_hat[n,m] = poisson_rng(lambda);
            Right_hat[n,m] = beta_binomial_rng(Total[n,m], alphas[n], betas[m]);
        }
    }
}
"""

In [18]:
sm_unknown1 = pystan.StanModel(model_code=binomial_on_2D_code_unknown1)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_44a39ace186659875e656c684038eec2 NOW.


In [19]:
fit_unknown1 = sm_unknown1.sampling(
    data=binomial_on_2D_dat, 
    iter=1000, 
    chains=4
)

In [20]:
dims = {"alphas":["river_distance"], 
        "betas":["sea_distance"], 
        "Total": ["river_distance", "sea_distance"], 
        "Right": ["river_distance", "sea_distance"], 
        "Total_hat": ["river_distance", "sea_distance"], 
        "Right_hat": ["river_distance", "sea_distance"], 
        "log_lik": ["river_distance", "sea_distance"]}
coords = {"river_distance":range(N), "sea_distance": range(M)}
idata_unknown1 = az.from_pystan(
    posterior=fit_unknown1,
    observed_data=['Total', 'Right'],
    posterior_predictive=['Total_hat', 'Right_hat'],
    log_likelihood="log_lik",
    coords=coords,
    dims=dims
)

In [21]:
az.loo(idata_unknown1)

/home/oriol/venvs/arviz-dev/lib/python3.6/site-packages/arviz/stats/stats.py:425: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.7 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO posterior are very different. This is more likely to happen with a non-robust model and highly influential observations.
  "Estimated shape parameter of Pareto distribution is greater than 0.7 for "


loo           54294.8
loo_se        5755.95
p_loo         513.177
warning          True
loo_scale    deviance
dtype: object

In [22]:
idata_unknown1.to_netcdf("binomial_on_2D_pystan_unknown1.nc")

'binomial_on_2D_pystan_unknown1.nc'

## Unknown number of inhabitants (version 2)
The fourth modelling option will be a variation on the first model, assuming that $N_{inhabitants}$ is not known, and modelling the participation as a poisson process. The difference with the third model is that lambda is decomposed into the sum of the x component and the y component. Hence, $votes(x,y) \sim \text{Poisson}\big(\lambda_1(x)+\lambda_2(y)\big)$ and $right(x,y) \sim BetaBinomial\Big(votes(x,y), \alpha(x),\beta(y)\Big)$.

In [23]:
binomial_on_2D_code_unknown2 = """
data {
    int<lower=1> N;     // num of x, or num of river_distance values
    int<lower=1> M;     // num of y, or num of sea_distance values

    int Total[N,M];
    int Right[N,M];
}

parameters {
    vector<lower=0>[N] alphas;     
    vector<lower=0>[M] betas;
    vector<lower=0>[N] lambda1;     
    vector<lower=0>[M] lambda2;
}

model {

    for (n in 1:N){
        for (m in 1:M){
            Total[n,m] ~ poisson(lambda1[n]+lambda2[m]);
            Right[n,m] ~ beta_binomial(Total[n,m], alphas[n], betas[m]);
        }
    }
}

generated quantities {
    real log_lik[N,M];
    real Total_hat[N,M];
    real Right_hat[N,M];
    
    for (n in 1:N){
        for (m in 1:M){
            log_lik[n,m] = poisson_lpmf(Total[n,m] | lambda1[n]+lambda2[m]) + 
                           beta_binomial_lpmf(Right[n,m] | Total[n,m], alphas[n], betas[m]);
            Total_hat[n,m] = poisson_rng(lambda1[n]+lambda2[m]);
            Right_hat[n,m] = beta_binomial_rng(Total[n,m], alphas[n], betas[m]);
        }
    }
}
"""

In [ ]:
sm_unknown2 = pystan.StanModel(model_code=binomial_on_2D_code_unknown2)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_87a34b0d590d41f713ec978839cbf17b NOW.


In [ ]:
fit_unknown2 = sm_unknown2.sampling(
    data=binomial_on_2D_dat, 
    iter=3000, 
    chains=8,
    control={"max_treedepth":15, "adapt_delta":.86}
)

In [ ]:
dims = {"alphas":["river_distance"], 
        "betas":["sea_distance"], 
        "lambda1":["river_distance"], 
        "lambda2":["sea_distance"],
        "Total": ["river_distance", "sea_distance"], 
        "Right": ["river_distance", "sea_distance"], 
        "Total_hat": ["river_distance", "sea_distance"], 
        "Right_hat": ["river_distance", "sea_distance"], 
        "log_lik": ["river_distance", "sea_distance"]}
coords = {"river_distance":range(N), "sea_distance": range(M)}
idata_unknown2 = az.from_pystan(
    posterior=fit_unknown2,
    observed_data=['Total', 'Right'],
    posterior_predictive=['Total_hat', 'Right_hat'],
    log_likelihood="log_lik",
    coords=coords,
    dims=dims
)

In [ ]:
az.loo(idata_unknown2)